<a href="https://colab.research.google.com/github/khai-likelion/data-ETL/blob/main/review_analysis_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

환경 설정 및 라이브러리 설치

In [ ]:
!pip -q install -U --force-reinstall \
  "transformers==4.44.2" \
  "accelerate==0.33.0" \
  "bitsandbytes==0.43.1"

# pipeline이 torchvision 때문에 죽는 케이스가 많아서, 텍스트 생성만 할 거면 torchvision은 과감히 제거해도 됨
!pip -q uninstall -y torchvision


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 146.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.3/566.3 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 131.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 1. 기존에 꼬여있을 수 있는 패키지 삭제 및 최신 버전 설치
!pip uninstall -y bitsandbytes triton
!pip install bitsandbytes triton accelerate transformers

# 2. (선택사항) 만약 위 방법으로도 안 된다면, bitsandbytes를 소스에서 직접 업데이트 하거나
# 특정 버전을 지정하는 것이 방법입니다. 보통은 최신 버전 설치로 해결됩니다.

Found existing installation: bitsandbytes 0.43.1
Uninstalling bitsandbytes-0.43.1:
  Successfully uninstalled bitsandbytes-0.43.1
Found existing installation: triton 3.6.0
Uninstalling triton-3.6.0:
  Successfully uninstalled triton-3.6.0
  Using cached triton-3.6.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (1.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 45.9 MB/s eta 0:00:00
Using cached triton-3.6.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (188.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.8.6 requires torchvision>=0.11, which is not installed.
timm 1.0.24 requires torchvision, which is not installed.
torchaudio 2.9.0+cu126 requires torch==2.9.0, but you have torch 2.10.0 which is incompatible.
fastai 2.8.6 requires torch<2.10,>=1.10, but you have torch 2.10.0 which is incompatib

In [ ]:
import transformers, accelerate
import bitsandbytes as bnb
print("transformers:", transformers.__version__)
print("accelerate:", accelerate.__version__)
print("bitsandbytes:", bnb.__version__)

transformers: 4.57.6
accelerate: 1.12.0
bitsandbytes: 0.49.1


구글 드라이브 마운트 및 데이터 로드

In [ ]:
import pandas as pd
import numpy as np
import json
import torch
import os
import re
from google.colab import drive
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# 1. 드라이브 마운트 및 경로 설정
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/likelion-khai'
checkpoint_path = os.path.join(base_path, 'mangwon_only_reviews_report.json')

# 2. 데이터 로드
reviews = pd.read_csv(os.path.join(base_path, 'mangwon_reviews.csv'), encoding='utf-8-sig', dtype={'ID': str})
mapped = pd.read_csv(os.path.join(base_path, 'mangwon_mapped_v2.csv'), encoding='cp949', dtype={'ID': str})

# 리뷰내용과 뱃지 통합 (맥락 강화)
reviews['종합리뷰'] = reviews.apply(lambda x: f"[{x['리뷰뱃지']}] {x['리뷰내용']}" if pd.notna(x['리뷰뱃지']) else x['리뷰내용'], axis=1)

df = pd.merge(reviews, mapped[['ID', '업종']], on='ID', how='left')
category_avg = (df.groupby('업종')['별점'].mean() / 5).to_dict()

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

모델 로드

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

torch.cuda.empty_cache()

model_name = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0},
    trust_remote_code=True,
    low_cpu_mem_usage=True
)

model.eval()
model.config.use_cache = False

def generate_text(messages, **kwargs):
    # 1. EXAONE-3.5 전용 채팅 템플릿 적용
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # 2. 입력을 GPU로 이동
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # 3. 모델 생성 (kwargs를 통해 do_sample, temperature 등을 유연하게 받음)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            **kwargs  # 여기서 do_sample, top_p 등이 처리됩니다
        )

    # 4. 결과 디코딩 및 반환
    input_length = inputs.input_ids.shape[1]
    return tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True)


분석 및 검증 함수 정의 (RAG 논리 적용)

In [ ]:
import json, re

def extract_first_json(text: str):
    if not text:
        return None

    t = re.sub(r"```(?:json)?", "", text)
    t = t.replace("```", "").strip()

    m = re.search(r"\{", t)
    if not m:
        return None

    start = m.start()
    dec = json.JSONDecoder()

    # 첫 JSON 객체만 파싱 (뒤에 뭐가 더 있어도 무시)
    try:
        obj, _ = dec.raw_decode(t[start:])
        return obj
    except json.JSONDecodeError:
        # 혹시 중간에 다른 '{'가 진짜 시작이면 재시도
        for mm in re.finditer(r"\{", t[start+1:]):
            s = start + 1 + mm.start()
            try:
                obj, _ = dec.raw_decode(t[s:])
                return obj
            except json.JSONDecodeError:
                continue
        return None


In [ ]:
# 4. 분석 함수
def generate_and_verify_report(store_data, reviews_list, benchmark):
    full_text = "\n".join([f"[{i}] {txt}" for i, txt in enumerate(reviews_list)])[:3500]

    system_prompt = """당신은 식당 리뷰 통계 분석가입니다.
주관적인 미사여구는 배제하고, 반드시 제공된 데이터의 '수치'와 '빈도'에 기반하여 JSON으로 답변하세요."""

    user_prompt = f"""
식당: {store_data['장소명']} (ID: {store_data['ID']})
데이터: {full_text}

[작성 지침]
1. critical_feedback: 리뷰 원문에서 불만사항을 추출하되, "리뷰 X번" 같은 번호는 절대 언급하지 마세요.
   - (나쁜 예: "리뷰 5번에서 양이 적다고 함")
   - (좋은 예: "전반적으로 양이 적다는 의견이 반복됨", "음식의 간이 세다는 피드백이 있음")
2. rag_context: 단순 수치 나열이 아니라, 식당의 전반적인 분위기와 특징을 포함한 '자연스러운 문장'으로 작성하세요.
   - 통계적 근거는 문장 안에 자연스럽게 녹여내세요. (예: "15건의 리뷰 중 대부분이 청결함을 언급하며 긍정적인 반응을 보였습니다.")

{{
  "store_id": "{store_data['ID']}",
  "store_name": "{store_data['장소명']}",
  "review_metrics": {{
    "overall_sentiment": {{ "score": 0.0, "label": "", "comparison": "" }},
    "feature_scores": {{
      "taste": {{ "score": 0.0, "label": "", "avg_score": {benchmark} }},
      "price_value": {{ "score": 0.0, "label": "", "avg_score": 0.58 }},
      "cleanliness": {{ "score": 0.0, "label": "", "avg_score": 0.72 }},
      "service": {{ "score": 0.0, "label": "", "avg_score": 0.68 }}
    }}
  }},
  "top_keywords": [],
  "critical_feedback": [],
  "rag_context": ""
}}"""

    try:
        messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}]
        # generate_text 호출 부분 수정
        res_txt = generate_text(
            messages,
            max_new_tokens=1024,  # 충분한 길이 확보
            do_sample=True,
            temperature=0.1,      # 너무 창의적이지 않게 (낮은 온도)
            top_p=0.9
        )

        # 1. JSON 영역 추출
        start_idx = res_txt.find('{')
        end_idx = res_txt.rfind('}') + 1
        if start_idx == -1: return None
        # 1. JSON 파싱 (첫 JSON만)
        parsed_json = extract_first_json(res_txt)
        if parsed_json is None or not isinstance(parsed_json, dict):
            raise ValueError("JSON 파싱 실패: 모델 출력이 JSON 단일 객체가 아님")


        # 2. [중요] template 정의 (객체 초기화)
        template = {
            "store_id": str(store_data['ID']),
            "store_name": store_data['장소명'],
            "analysis_date": "2026-01-29",
            "review_metrics": {
                "overall_sentiment": {"score": 0.0, "label": "보통", "comparison": "평균 수준"},
                "feature_scores": {
                    "taste": {"score": 0.0, "label": "보통", "avg_score": benchmark},
                    "price_value": {"score": 0.0, "label": "보통", "avg_score": 0.58},
                    "cleanliness": {"score": 0.0, "label": "보통", "avg_score": 0.72},
                    "service": {"score": 0.0, "label": "보통", "avg_score": 0.68}
                }
            },
            "top_keywords": [],
            "critical_feedback": [],
            "rag_context": ""
        }

        # 3. 데이터 병합
        if "review_metrics" in parsed_json:
            template["review_metrics"].update(parsed_json["review_metrics"])
        template["top_keywords"] = parsed_json.get("top_keywords", [])
        template["rag_context"] = parsed_json.get("rag_context", "수치 근거 부족")

        # 4. Critical Feedback 검증 및 담기
        raw_f = parsed_json.get('critical_feedback', [])
        verified = []
        # 검증 부분 수정
        for f in raw_f:
            f_clean = f.strip()
            if len(f_clean) >= 2: # 글자수 제한 완화
                # 리뷰 리스트 중 하나라도 해당 단어를 포함하고 있다면 통과
                if any(f_clean in r for r in reviews_list):
                    verified.append(f_clean)
                else:
                    # 원문과 완벽히 일치하지 않아도 모델이 추출한 핵심 키워드라면 일단 유지 (선택 사항)
                    verified.append(f_clean)

        template['critical_feedback'] = verified
        return template

    except Exception as e:
        print(f"❌ 분석 실패 ({store_data['장소명']}): {e}")
        return None

전체 순회 및 체크포인트 저장

In [ ]:
# # 5. 실행 루프
# final_reports = []
# if os.path.exists(checkpoint_path):
#     with open(checkpoint_path, 'r', encoding='utf-8') as f:
#         final_reports = json.load(f)
# analyzed_ids = {str(r.get('store_id')) for r in final_reports}

# for sid in df['ID'].unique():
#     if str(sid) in analyzed_ids: continue

#     s_df = df[df['ID'] == sid]
#     s_info = s_df.iloc[0]

#     # 리뷰가 150개 이상이면 최신순/랜덤으로 100개만 샘플링 (속도 및 메모리 보호)
#     r_list = s_df['종합리뷰'].dropna().tolist()
#     if len(r_list) > 100:
#         r_list = r_list[:100]

#     print(f"⏳ 분석 시작: {s_info['식당명']} (리뷰 {len(r_list)}개)")

#     report = generate_and_verify_report(s_info, r_list, 0.7)

#     if report:
#         final_reports.append(report)
#         with open(checkpoint_path, 'w', encoding='utf-8') as f:
#             json.dump(final_reports, f, ensure_ascii=False, indent=4)
#         print(f"✅ 저장 성공: {s_info['식당명']}")

#     torch.cuda.empty_cache()

⏳ 분석 시작: 일등식당 (리뷰 100개)
✅ 저장 성공: 일등식당
⏳ 분석 시작: 망원시장손칼국수 (리뷰 100개)
✅ 저장 성공: 망원시장손칼국수
⏳ 분석 시작: 청어람 망원점 (리뷰 100개)
✅ 저장 성공: 청어람 망원점
⏳ 분석 시작: 순대일번지 (리뷰 100개)
✅ 저장 성공: 순대일번지
⏳ 분석 시작: 육장 (리뷰 100개)
✅ 저장 성공: 육장
⏳ 분석 시작: 고향집 (리뷰 64개)
✅ 저장 성공: 고향집
⏳ 분석 시작: 진평925 (리뷰 32개)
✅ 저장 성공: 진평925
⏳ 분석 시작: 한강껍데기 (리뷰 100개)
✅ 저장 성공: 한강껍데기
⏳ 분석 시작: 어수선 (리뷰 27개)
✅ 저장 성공: 어수선
⏳ 분석 시작: 오복수산시장 (리뷰 18개)
✅ 저장 성공: 오복수산시장
⏳ 분석 시작: 현정이네철판두루치기 본점 (리뷰 40개)
✅ 저장 성공: 현정이네철판두루치기 본점
⏳ 분석 시작: 원조청기와숯불갈비 (리뷰 46개)
✅ 저장 성공: 원조청기와숯불갈비
⏳ 분석 시작: 망원동돼지국밥 (리뷰 79개)
✅ 저장 성공: 망원동돼지국밥
⏳ 분석 시작: 몽골생소금구이 (리뷰 62개)
✅ 저장 성공: 몽골생소금구이
⏳ 분석 시작: 퍼줄래대박집 (리뷰 46개)
✅ 저장 성공: 퍼줄래대박집
⏳ 분석 시작: 할머니빈대떡 망원본점 (리뷰 57개)
✅ 저장 성공: 할머니빈대떡 망원본점
⏳ 분석 시작: 하심정 (리뷰 28개)
✅ 저장 성공: 하심정
⏳ 분석 시작: 망원동고기집 망원본점 (리뷰 62개)
✅ 저장 성공: 망원동고기집 망원본점
⏳ 분석 시작: 산청엔흑돼지 서울1호점 (리뷰 91개)
✅ 저장 성공: 산청엔흑돼지 서울1호점
⏳ 분석 시작: 청어람 2호점 (리뷰 41개)
✅ 저장 성공: 청어람 2호점
⏳ 분석 시작: 정각 서울망원본점 (리뷰 100개)
✅ 저장 성공: 정각 서울망원본점
⏳ 분석 시작: 또또칼국수 (리뷰 53개)
✅ 저장 성공: 또또칼국수
⏳ 분석 시작: 청아라생선구이 망원점 (리뷰 50개)
✅ 저장 성공: 청아라생선구이 망원점
⏳ 분석 시작:

블로그 데이터 1차 추가

In [ ]:
import json
import os
import pandas as pd

# 1. 파일 로드
json_path = os.path.join(base_path, 'NaverMap_Final_Extracted.json')
with open(json_path, 'r', encoding='utf-8') as f:
    blog_data = json.load(f)

# 2. 블로그 데이터를 담을 리스트 (기존 df와 유사한 구조로 생성)
blog_rows = []
for store in blog_data:
    store_name = store.get('Name')
    # 식당별로 여러 명이 쓴 리뷰들을 리스트로 수집
    reviews = [rev.get('Content', '') for rev in store.get('Reviews', []) if rev.get('Content')]

    if reviews:
        blog_rows.append({
            'ID': f"BLOG_{store_name}", # ID가 없는 블로그용 임시 ID
            '장소명': store_name,
            '리뷰리스트': reviews
        })

print(f"✅ 블로그 데이터 로드 완료: {len(blog_rows)}개 식당")

✅ 블로그 데이터 로드 완료: 112개 식당


In [ ]:
import json
import os
import pandas as pd
import torch

# 1. 환경 설정 및 데이터 로드
checkpoint_path = os.path.join(base_path, 'mangwon_blog_report.json')
mapped_df = pd.read_csv(os.path.join(base_path, 'mangwon_mapped_v2.csv'), encoding='cp949')

with open(os.path.join(base_path, 'NaverMap_Final_Extracted.json'), 'r', encoding='utf-8-sig') as f:
    blog_data = json.load(f)

# 2. 기존 결과 로드 및 분석 완료 ID 파악
final_reports = []
if os.path.exists(checkpoint_path):
    with open(checkpoint_path, 'r', encoding='utf-8-sig') as f:
        final_reports = json.load(f)
analyzed_ids = {str(r.get('store_id')) for r in final_reports}

# 3. 분석 루프 및 결과 수집
not_found_in_mapping = []  # 매핑 파일에 이름이 없는 경우
analysis_failed = []      # 모델 분석 중 에러난 경우

for store in blog_data:
    raw_name = store.get('Name')

    # [ID 매칭 로직] mapped_df에서 해당 식당의 ID 찾기
    # 양쪽 공백 제거 및 완전 일치 기준 (필요시 str.contains 사용 가능)
    match = mapped_df[mapped_df['장소명'].str.strip() == raw_name.strip()]

    if match.empty:
        not_found_in_mapping.append(raw_name)
        continue

    sid = str(match.iloc[0]['ID'])
    real_name = match.iloc[0]['장소명'] # 매핑된 공식 명칭 사용

    # 이미 분석했다면 스킵
    if sid in analyzed_ids:
        continue

    # 여러 사람이 쓴 리뷰 리스트화
    r_list = [rev.get('Content', '') for rev in store.get('Reviews', []) if rev.get('Content')]

    # 데이터가 아예 없는 경우 스킵
    if not r_list:
        continue

    # 리뷰가 너무 많으면 100개 샘플링 (기존 로직 유지)
    if len(r_list) > 100:
        r_list = r_list[:100]

    print(f"⏳ 블로그 분석 시작: {real_name} (ID: {sid}, 리뷰 {len(r_list)}개)")

    # 기존 함수 그대로 호출 (s_info 형식 맞춰서 전달)
    s_info = {'ID': sid, '장소명': real_name}
    report = generate_and_verify_report(s_info, r_list, 0.7)

    if report:
        final_reports.append(report)
        # 실시간 저장 (체크포인트)
        with open(checkpoint_path, 'w', encoding='utf-8') as f:
            json.dump(final_reports, f, ensure_ascii=False, indent=4)
        print(f"✅ 저장 완료: {real_name}")
    else:
        analysis_failed.append(real_name)

    torch.cuda.empty_cache()

# 4. 최종 분석 현황 리포트
print("\n" + "="*50)
print("📊 최종 분석 정산 리포트")
print(f"1. 성공적으로 분석된 식당: {len(final_reports)}개")
print(f"2. 매핑 파일(CSV)에서 ID를 찾지 못한 식당: {len(not_found_in_mapping)}개")
if not_found_in_mapping:
    print(f"   - 누락 목록: {', '.join(not_found_in_mapping[:10])}...")
print(f"3. 모델 분석 과정에서 실패한 식당: {len(analysis_failed)}개")
if analysis_failed:
    print(f"   - 실패 목록: {', '.join(analysis_failed)}")
print("="*50)

⏳ 블로그 분석 시작: 초월양곱창 (ID: 1134437672, 리뷰 2개)
✅ 저장 완료: 초월양곱창
⏳ 블로그 분석 시작: 키친갈매기 (ID: 1373294203, 리뷰 2개)
✅ 저장 완료: 키친갈매기
⏳ 블로그 분석 시작: 오시 망원본점 (ID: 863823354, 리뷰 3개)
✅ 저장 완료: 오시 망원본점
⏳ 블로그 분석 시작: 정든그릇 망원점 (ID: 192598815, 리뷰 3개)
✅ 저장 완료: 정든그릇 망원점
⏳ 블로그 분석 시작: 다이치 (ID: 396604563, 리뷰 3개)
✅ 저장 완료: 다이치
⏳ 블로그 분석 시작: 티노마드모리 (ID: 1546506535, 리뷰 2개)
✅ 저장 완료: 티노마드모리
⏳ 블로그 분석 시작: 벚꽃스시 망원점 (ID: 690650568, 리뷰 3개)
✅ 저장 완료: 벚꽃스시 망원점
⏳ 블로그 분석 시작: 설경 (ID: 1673857593, 리뷰 3개)
✅ 저장 완료: 설경
⏳ 블로그 분석 시작: 고양꼬치 마포구청점 (ID: 946372629, 리뷰 3개)
✅ 저장 완료: 고양꼬치 마포구청점
⏳ 블로그 분석 시작: 무어스 (ID: 805235919, 리뷰 3개)
✅ 저장 완료: 무어스
⏳ 블로그 분석 시작: 유어다이닝 (ID: 1594402645, 리뷰 2개)
✅ 저장 완료: 유어다이닝
⏳ 블로그 분석 시작: 웨얼이즈누들 (ID: 612300151, 리뷰 3개)
✅ 저장 완료: 웨얼이즈누들
⏳ 블로그 분석 시작: 노모어피자 마포구청점 (ID: 988313618, 리뷰 3개)
✅ 저장 완료: 노모어피자 마포구청점
⏳ 블로그 분석 시작: 논드라이 (ID: 213806746, 리뷰 3개)
✅ 저장 완료: 논드라이
⏳ 블로그 분석 시작: 쿠루리 (ID: 770918827, 리뷰 3개)
✅ 저장 완료: 쿠루리
⏳ 블로그 분석 시작: 선데이카리오카 (ID: 2127662487, 리뷰 3개)
✅ 저장 완료: 선데이카리오카
⏳ 블로그 분석 시작: 라오삐약 (ID: 539411343, 리뷰 2개)
✅ 저장 완료: 라오삐약
⏳

중복ID 확인

In [ ]:
import json
import os
import pandas as pd

# 1. 파일 경로 설정
blog_report_path = os.path.join(base_path, 'mangwon_blog_report.json')
review_report_path = os.path.join(base_path, 'mangwon_only_reviews_report.json')
mapped_csv_path = os.path.join(base_path, 'mangwon_mapped_v2.csv')

# 2. 리포트 로드 및 ID-이름 매핑 생성 함수
def get_report_dict(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8-sig') as f:
            data = json.load(f)
            # {ID: 식당명} 딕셔너리 생성
            return {str(item.get('store_id')): item.get('store_name') for item in data}
    return {}

# 3. 데이터 로드
blog_dict = get_report_dict(blog_report_path)
review_dict = get_report_dict(review_report_path)
mapped_df = pd.read_csv(mapped_csv_path, encoding='cp949')

# 4. 중복 ID 찾기 (교집합)
duplicate_ids = set(blog_dict.keys()) & set(review_dict.keys())

# 5. 결과 출력
print("="*50)
print(f"🔍 리포트 간 중복 식당 확인 (총 {len(duplicate_ids)}개)")
print("-" * 50)

if not duplicate_ids:
    print("중복된 식당 ID가 없습니다.")
else:
    # 중복된 식당의 ID와 이름을 표 형태로 출력
    dup_list = []
    for sid in duplicate_ids:
        # 블로그 리포트나 리뷰 리포트 중 이름이 있는 곳에서 가져옴
        name = blog_dict.get(sid) or review_dict.get(sid)
        dup_list.append({'ID': sid, '식당명': name})

    df_dup = pd.DataFrame(dup_list)
    print(df_dup.to_string(index=False))

print("-" * 50)
print("💡 팁: 중복된 식당은 일반 리뷰와 블로그 데이터가 모두 존재합니다.")
print("최종 합본을 만들 때 '리뷰 개수가 더 많은 쪽'을 선택하거나 데이터를 병합할 수 있습니다.")
print("="*50)

🔍 리포트 간 중복 식당 확인 (총 5개)
--------------------------------------------------
        ID             식당명
  11246724             하심정
2100908584              시우
1907652089 땅스부대찌개 망원월드컵시장점
1559821182             아루감
1088931092             망원도
--------------------------------------------------
💡 팁: 중복된 식당은 일반 리뷰와 블로그 데이터가 모두 존재합니다.
최종 합본을 만들 때 '리뷰 개수가 더 많은 쪽'을 선택하거나 데이터를 병합할 수 있습니다.


누락ID 확인

In [ ]:
import json
import os
import pandas as pd

# 1. 파일 경로 설정
blog_report_path = os.path.join(base_path, 'mangwon_blog_report.json')
review_report_path = os.path.join(base_path, 'mangwon_only_reviews_report.json')
mapped_csv_path = os.path.join(base_path, 'mangwon_mapped_v2.csv')

# 2. 리포트 데이터 로드 함수
def load_json_ids(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            # 각 리포트의 store_id를 추출 (문자열로 통일)
            return {str(item.get('store_id')) for item in data}
    return set()

# 3. 데이터 불러오기
blog_ids = load_json_ids(blog_report_path)
review_ids = load_json_ids(review_report_path)
mapped_df = pd.read_csv(mapped_csv_path, encoding='cp949')

# 4. ID 통합 및 중복 제거
# 두 리포트에 모두 있는 식당도 있으므로 합집합(union)을 구합니다.
total_analyzed_ids = blog_ids | review_ids
all_mapped_ids = set(mapped_df['ID'].astype(str).unique())

# 5. 결과 계산
intersect_ids = total_analyzed_ids & all_mapped_ids  # 실제로 매핑 파일에 존재하는 분석된 ID
missing_ids = all_mapped_ids - total_analyzed_ids     # 매핑 파일에는 있지만 리포트에는 없는 ID

# 6. 리포트 출력
print("="*50)
print("📊 분석 결과 통합 및 누락 확인")
print(f"1. 블로그 리포트 ID 개수: {len(blog_ids)}개")
print(f"2. 일반 리뷰 리포트 ID 개수: {len(review_ids)}개")
print(f"3. 중복 제외 통합 분석 완료 ID: {len(total_analyzed_ids)}개")
print(f"4. mapped_v2.csv 전체 식당 개수: {len(all_mapped_ids)}개")
print("-" * 50)

if len(missing_ids) == 0:
    print("✅ 축하합니다! 모든 식당(ID)이 리포트에 포함되어 있습니다.")
else:
    print(f"⚠️ 누락된 식당 발견: {len(missing_ids)}개")
    print("📍 누락된 식당 목록 (상위 10개):")
    # 누락된 ID의 실제 이름을 확인하기 위해 다시 매핑
    missing_names = mapped_df[mapped_df['ID'].astype(str).isin(missing_ids)][['ID', '장소명']]
    for _, row in missing_names.iterrows():
        print(f"   - ID: {row['ID']} | 이름: {row['장소명']}")
print("="*50)

📊 분석 결과 통합 및 누락 확인
1. 블로그 리포트 ID 개수: 74개
2. 일반 리뷰 리포트 ID 개수: 650개
3. 중복 제외 통합 분석 완료 ID: 719개
4. mapped_v2.csv 전체 식당 개수: 756개
--------------------------------------------------
⚠️ 누락된 식당 발견: 37개
📍 누락된 식당 목록 (상위 10개):
   - ID: 1100082903 | 이름: 바삭마차 망원본점
   - ID: 2119780640 | 이름: 게뜨망하우스
   - ID: 1063033896 | 이름: 어수선회&초밥
   - ID: 65852876 | 이름: BBQ 한강버스망원선착장점
   - ID: 211364003 | 이름: 불만있는치킨
   - ID: 93122129 | 이름: 망원장터국밥
   - ID: 2098634499 | 이름: 식해
   - ID: 1873776 | 이름: 촌
   - ID: 684089354 | 이름: 얼쑤
   - ID: 1065846269 | 이름: 본가안동찜닭닭도리탕
   - ID: 1997011697 | 이름: 황제정육식당&수제돼지갈비
   - ID: 1374582458 | 이름: 구구족 망원점
   - ID: 1535960191 | 이름: 서교주담 망원점
   - ID: 1639736108 | 이름: 망원리 황태 콩나물해장국
   - ID: 1075250749 | 이름: 노가리랑닭발
   - ID: 27336512 | 이름: 샘밭골
   - ID: 2032582194 | 이름: 대장군부속구이
   - ID: 1128510180 | 이름: 타코장인24시
   - ID: 1691318306 | 이름: 더바이글
   - ID: 911552927 | 이름: 디망쉬키친
   - ID: 1879823956 | 이름: 마닐다과
   - ID: 308774273 | 이름: 밀리언스
   - ID: 117343217 | 이름: 이자카야제때 망원점
   - ID: 1748676286 | 이

병합

In [ ]:
import json
import os

# 1. 파일 경로 설정
blog_report_path = os.path.join(base_path, 'mangwon_blog_report.json')
review_report_path = os.path.join(base_path, 'mangwon_only_reviews_report.json')
final_merge_path = os.path.join(base_path, 'mangwon_final_integrated_report.json')

# 2. 데이터 로드
def load_json(path):
    if os.path.exists(path):
        with open(path, 'r', encoding='utf-8') as f:
            return json.load(f)
    return []

blog_data = load_json(blog_report_path)
review_data = load_json(review_report_path)

# 3. 병합 전략 수행 (Kakao 우선)
merged_dict = {}

# [Step 1] 블로그 데이터를 먼저 채우기
for report in blog_data:
    sid = str(report.get('store_id'))
    merged_dict[sid] = report

# [Step 2] Kakao 일반 리뷰 데이터로 덮어쓰기 (중복 시 Kakao가 최종본이 됨)
duplicate_count = 0
for report in review_data:
    sid = str(report.get('store_id'))

    if sid in merged_dict:
        # 이미 블로그 데이터가 있는 경우 Kakao로 교체
        duplicate_count += 1
        # 필요하다면 여기서 어떤 데이터가 교체되는지 로그를 남길 수 있습니다.

    merged_dict[sid] = report

# 4. 리스트로 변환 및 최종 저장
final_integrated_list = list(merged_dict.values())

with open(final_merge_path, 'w', encoding='utf-8') as f:
    json.dump(final_integrated_list, f, ensure_ascii=False, indent=4)

# 5. 최종 결과 확인
print("="*50)
print("✅ Kakao 일반 리뷰 우선 병합 완료")
print(f"- 총 통합 식당 개수: {len(final_integrated_list)}개")
print(f"- 중복되어 Kakao로 덮어쓴 식당: {duplicate_count}개")
print(f"- 블로그 데이터로만 채워진 식당: {len(blog_data) - duplicate_count}개")
print("-" * 50)
print(f"📍 저장 위치: {final_merge_path}")
print("="*50)

✅ Kakao 일반 리뷰 우선 병합 완료
- 총 통합 식당 개수: 719개
- 중복되어 Kakao로 덮어쓴 식당: 5개
- 블로그 데이터로만 채워진 식당: 69개
--------------------------------------------------
📍 저장 위치: /content/drive/MyDrive/likelion-khai/mangwon_final_integrated_report.json


블로그 데이터 2차 추가

In [ ]:
import json
import os
import pandas as pd

# 1. 경로 설정
# base_path = "본인의 경로로 설정하세요"
checkpoint_path = os.path.join(base_path, 'mangwon_blog_extracted_reviews.json')
csv_path = os.path.join(base_path, 'check_list.csv')
json_input_path = os.path.join(base_path, 'ch2.json')

# 2. 데이터 로드
# ID가 지수표기법으로 변하지 않도록 문자열(str)로 읽기
mapped_df = pd.read_csv(csv_path, encoding='cp949', dtype={'ID': str})

with open(json_input_path, 'r', encoding='utf-8-sig') as f:
    blog_data = json.load(f)

# 3. 데이터 매칭 및 추출
extracted_results = []
not_found_count = 0

for store in blog_data:
    raw_name = (store.get('Name') or "").strip()

    # [매칭 로직] CSV의 'Name' 컬럼과 json의 'Name'이 일치하는 행 찾기
    match = mapped_df[mapped_df['Name'].str.strip() == raw_name]

    # 직접 일치가 없으면 SearchKeyword로 한 번 더 확인 (유연한 매칭)
    if match.empty:
        match = mapped_df[mapped_df['SearchKeyword'].str.contains(raw_name, na=False) if raw_name else False]

    if not match.empty:
        sid = str(match.iloc[0]['ID'])
        real_name = match.iloc[0]['Name']

        # 리뷰 리스트 추출 (내용이 있는 것만)
        r_list = [rev.get('Content', '') for rev in store.get('Reviews', []) if rev.get('Content')]

        if r_list:
            extracted_results.append({
                "store_id": sid,
                "store_name": real_name,
                "address": match.iloc[0]['Address'],
                "reviews": r_list
            })
            print(f"✅ 매칭 성공: {real_name} (ID: {sid}, 리뷰 {len(r_list)}개)")
    else:
        not_found_count += 1

# 4. 결과 저장
with open(checkpoint_path, 'w', encoding='utf-8') as f:
    json.dump(extracted_results, f, ensure_ascii=False, indent=4)

# 5. 최종 결과 확인
print("\n" + "="*50)
print(f"📊 추출 완료 리포트")
print(f"1. 리뷰가 포함된 식당 개수: {len(extracted_results)}개")
print(f"2. CSV 매핑에 실패한 식당 개수: {not_found_count}개")
print(f"📍 저장 위치: {checkpoint_path}")
print("="*50)

✅ 매칭 성공: 장터국밥 (ID: 1637957216, 리뷰 2개)
✅ 매칭 성공: 식해 한식바다주점 (ID: 2098634499, 리뷰 2개)
✅ 매칭 성공: 이자카야 촌 (ID: 1873776, 리뷰 3개)
✅ 매칭 성공: 얼쑤 망원 (ID: 684089354, 리뷰 3개)
✅ 매칭 성공: 황제정육식당&수제돼지갈비 (ID: 1997011697, 리뷰 1개)
✅ 매칭 성공: 서교주담 망원 (ID: 1535960191, 리뷰 3개)
✅ 매칭 성공: 더 바이글 (ID: 1691318306, 리뷰 2개)
✅ 매칭 성공: 디망쉬 키친 (ID: 911552927, 리뷰 3개)
✅ 매칭 성공: 마닐다과 화과자공방 (ID: 1879823956, 리뷰 3개)
✅ 매칭 성공: 이자카야 제때 망원점 (ID: 117343217, 리뷰 2개)
✅ 매칭 성공: Alive214 (ID: 1558684636, 리뷰 2개)
✅ 매칭 성공: 나우올네버 (ID: 2028626364, 리뷰 3개)
✅ 매칭 성공: 낮도깨비 밤도깨비 (ID: 1996118092, 리뷰 2개)
✅ 매칭 성공: 프뤼떼마지 본점작업실 (ID: 616856088, 리뷰 3개)
✅ 매칭 성공: 고미푸딩 망원점 (ID: 630795566, 리뷰 3개)

📊 추출 완료 리포트
1. 리뷰가 포함된 식당 개수: 15개
2. CSV 매핑에 실패한 식당 개수: 28개
📍 저장 위치: /content/drive/MyDrive/likelion-khai/mangwon_blog_extracted_reviews.json


In [ ]:
import json
import os
import pandas as pd
from difflib import SequenceMatcher

# 유사도 계산 함수 (0~1 사이 점수)
def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

# 1. 파일 로드
csv_path = os.path.join(base_path, 'check_list.csv')
json_input_path = os.path.join(base_path, 'ch2.json')
output_path = os.path.join(base_path, 'mangwon_final_extracted_reviews.json')

mapped_df = pd.read_csv(csv_path, encoding='cp949', dtype={'ID': str})
with open(json_input_path, 'r', encoding='utf-8-sig') as f:
    blog_data = json.load(f)

final_extracted = []
matched_ids = set()

# 2. 매칭 로직 (CSV 기준 루프 - 23개를 다 채우기 위함)
for _, row in mapped_df.iterrows():
    target_id = str(row['ID'])
    target_name = str(row['Name']).strip()
    search_key = str(row['SearchKeyword']).strip()

    best_match_store = None
    max_score = 0

    for store in blog_data:
        json_name = (store.get('Name') or "").strip()

        # 이름 유사도 측정 (예: '장터국밥' vs '망원장터국밥')
        score = similarity(target_name, json_name)

        # 포함 관계일 경우 가점 (예: '식해' vs '식해 한식바다주점')
        if target_name in json_name or json_name in target_name:
            score += 0.2

        if score > max_score:
            max_score = score
            best_match_store = store

    # 점수가 0.6 이상인 경우에만 매칭 성공으로 간주
    if max_score >= 0.6 and best_match_store:
        reviews = [rev.get('Content', '') for rev in best_match_store.get('Reviews', []) if rev.get('Content')]

        if reviews:
            final_extracted.append({
                "store_id": target_id,
                "store_name": target_name,
                "naver_match_name": best_match_store.get('Name'),
                "reviews": reviews
            })
            matched_ids.add(target_id)
            print(f"🎯 매칭 성공: {target_name} <- {best_match_store.get('Name')} (Score: {max_score:.2f})")

# 3. 결과 저장
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(final_extracted, f, ensure_ascii=False, indent=4)

# 4. 최종 정산
print("\n" + "="*50)
print(f"📊 최종 추출 결과")
print(f"- 타겟 식당 수: {len(mapped_df)}개")
print(f"- 실제 매칭 및 리뷰 추출 성공: {len(final_extracted)}개")
if len(mapped_df) > len(final_extracted):
    missing = set(mapped_df['Name']) - {x['store_name'] for x in final_extracted}
    print(f"- 누락된 식당: {list(missing)}")
print("="*50)

🎯 매칭 성공: 장터국밥 <- 장터국밥 (Score: 1.20)
🎯 매칭 성공: 식해 한식바다주점 <- 식해 한식바다주점 (Score: 1.20)
🎯 매칭 성공: 이자카야 촌 <- 이자카야 촌 (Score: 1.20)
🎯 매칭 성공: 얼쑤 망원 <- 얼쑤 망원 (Score: 1.20)
🎯 매칭 성공: 황제정육식당&수제돼지갈비 <- 황제정육식당&수제돼지갈비 (Score: 1.20)
🎯 매칭 성공: 서교주담 망원 <- 서교주담 망원 (Score: 1.20)
🎯 매칭 성공: 더 바이글 <- 더 바이글 (Score: 1.20)
🎯 매칭 성공: 디망쉬 키친 <- 디망쉬 키친 (Score: 1.20)
🎯 매칭 성공: 마닐다과 화과자공방 <- 마닐다과 화과자공방 (Score: 1.20)
🎯 매칭 성공: 이자카야 제때 망원점 <- 이자카야 제때 망원점 (Score: 1.20)
🎯 매칭 성공: Alive214 <- Alive214 (Score: 1.20)
🎯 매칭 성공: 나우올네버 <- 나우올네버 (Score: 1.20)
🎯 매칭 성공: 낮도깨비 밤도깨비 <- 낮도깨비 밤도깨비 (Score: 1.20)
🎯 매칭 성공: 프뤼떼마지 본점작업실 <- 프뤼떼마지 본점작업실 (Score: 1.20)
🎯 매칭 성공: 고미푸딩 망원점 <- 고미푸딩 망원점 (Score: 1.20)

📊 최종 추출 결과
- 타겟 식당 수: 23개
- 실제 매칭 및 리뷰 추출 성공: 15개
- 누락된 식당: ['망원리황태콩나물해장국', '오시오', '오켄스서울 망원익스프레스', '샘밭골', '망원옥상', '카페게이트 망원한강점', '대장군 부속구이', '구구족 망원점']


In [ ]:
import json
import os
import pandas as pd
import torch
from difflib import SequenceMatcher

# 1. 경로 설정 (사용자 환경에 맞게 수정)
# base_path = "/content/drive/MyDrive/..."
checkpoint_path = os.path.join(base_path, 'mangwon_blog_report2.json')
csv_path = os.path.join(base_path, 'check_list.csv')
json_input_path = os.path.join(base_path, 'ch2.json')

# 2. 데이터 로드 및 초기화
# ID를 문자열로 읽어 지수 표기법 방지
mapped_df = pd.read_csv(csv_path, encoding='cp949', dtype={'ID': str})
with open(json_input_path, 'r', encoding='utf-8-sig') as f:
    blog_data = json.load(f)

# 기존 분석 결과 로드 (중복 분석 방지)
final_reports = []
if os.path.exists(checkpoint_path):
    with open(checkpoint_path, 'r', encoding='utf-8-sig') as f:
        final_reports = json.load(f)
analyzed_ids = {str(r.get('store_id')) for r in final_reports}

# 3. 유사도 매칭 함수 (가점 포함)
def get_similarity(target, source):
    t_clean = str(target).replace(" ", "")
    s_clean = str(source).replace(" ", "")
    score = SequenceMatcher(None, t_clean, s_clean).ratio()
    if t_clean in s_clean or s_clean in t_clean:
        score += 0.2
    return score

# 4. 분석 루프 실행
# CSV에 명시된 23개 타겟 식당을 기준으로 순회
for _, row in mapped_df.iterrows():
    sid = str(row['ID'])
    target_name = str(row['Name']).strip()

    # 이미 분석된 ID는 스킵
    if sid in analyzed_ids:
        continue

    # [매칭 로직] ch2.json 내에서 최적의 식당 찾기
    best_match_store = None
    max_score = 0

    for store in blog_data:
        json_name = (store.get('Name') or "").strip()
        score = get_similarity(target_name, json_name)

        if score > max_score:
            max_score = score
            best_match_store = store

    # 임계값 0.7 이상인 경우에만 분석 진행
    if max_score >= 0.7 and best_match_store:
        # 리뷰 리스트 추출 (Content가 있는 항목만)
        r_list = [rev.get('Content', '') for rev in best_match_store.get('Reviews', []) if rev.get('Content')]

        if not r_list:
            continue

        # 리뷰 샘플링 (최대 100개 제한)
        if len(r_list) > 100:
            r_list = r_list[:100]

        print(f"⏳ 분석 시작: {target_name} (ID: {sid}, 매칭명: {best_match_store.get('Name')}, 리뷰 {len(r_list)}개)")

        # 기존 제공된 분석 함수 호출
        s_info = {'ID': sid, '장소명': target_name}
        report = generate_and_verify_report(s_info, r_list, 0.7)

        if report:
            # store_id를 명시적으로 저장하여 일치성 보장
            report['store_id'] = sid
            final_reports.append(report)

            # 분석마다 실시간 저장 (체크포인트 저장)
            with open(checkpoint_path, 'w', encoding='utf-8') as f:
                json.dump(final_reports, f, ensure_ascii=False, indent=4)

            analyzed_ids.add(sid)
            print(f"✅ 저장 성공: {target_name}")
        else:
            print(f"❌ 분석 실패/반환값 없음: {target_name}")

        # GPU 메모리 관리
        torch.cuda.empty_cache()

print("\n" + "="*50)
print(f"📊 최종 분석 완료: 총 {len(final_reports)}개 리포트 생성")
print("="*50)

⏳ 분석 시작: 장터국밥 (ID: 93122129, 매칭명: 장터국밥, 리뷰 2개)
✅ 저장 성공: 장터국밥

📊 최종 분석 완료: 총 16개 리포트 생성


In [ ]:
import json
import os

# 1. 파일 경로 설정
blog_report_path = os.path.join(base_path, 'mangwon_blog_report2.json')
review_report_path = os.path.join(base_path, 'mangwon_final_integrated_report.json')
final_merge_path = os.path.join(base_path, 'mangwon_final_final_integrated_report.json')

# 2. 데이터 로드
def load_json(path):
    if os.path.exists(path):
        with open(path, 'r', encoding='utf-8') as f:
            return json.load(f)
    return []

blog_data = load_json(blog_report_path)
review_data = load_json(review_report_path)

# 3. 병합 전략 수행 (Kakao 우선)
merged_dict = {}

# [Step 1] 블로그 데이터를 먼저 채우기
for report in blog_data:
    sid = str(report.get('store_id'))
    merged_dict[sid] = report

# [Step 2] Kakao 일반 리뷰 데이터로 덮어쓰기 (중복 시 Kakao가 최종본이 됨)
duplicate_count = 0
for report in review_data:
    sid = str(report.get('store_id'))

    if sid in merged_dict:
        # 이미 블로그 데이터가 있는 경우 Kakao로 교체
        duplicate_count += 1
        # 필요하다면 여기서 어떤 데이터가 교체되는지 로그를 남길 수 있습니다.

    merged_dict[sid] = report

# 4. 리스트로 변환 및 최종 저장
final_integrated_list = list(merged_dict.values())

with open(final_merge_path, 'w', encoding='utf-8') as f:
    json.dump(final_integrated_list, f, ensure_ascii=False, indent=4)

# 5. 최종 결과 확인
print("="*50)
print("✅ Kakao 일반 리뷰 우선 병합 완료")
print(f"- 총 통합 식당 개수: {len(final_integrated_list)}개")
print(f"- 중복되어 Kakao로 덮어쓴 식당: {duplicate_count}개")
print(f"- 블로그 데이터로만 채워진 식당: {len(blog_data) - duplicate_count}개")
print("-" * 50)
print(f"📍 저장 위치: {final_merge_path}")
print("="*50)

✅ Kakao 일반 리뷰 우선 병합 완료
- 총 통합 식당 개수: 734개
- 중복되어 Kakao로 덮어쓴 식당: 1개
- 블로그 데이터로만 채워진 식당: 15개
--------------------------------------------------
📍 저장 위치: /content/drive/MyDrive/likelion-khai/mangwon_final_final_integrated_report.json


In [ ]:
import json
import os

# 1. 파일 경로 (제공해주신 경로 유지)
blog_report_path = os.path.join(base_path, 'mangwon_blog_report2.json')
review_report_path = os.path.join(base_path, 'mangwon_final_integrated_report.json')
final_merge_path = os.path.join(base_path, 'mangwon_final_final_integrated_report.json')

# 2. 데이터 로드
blog_data = load_json(blog_report_path)
review_data = load_json(review_report_path)

merged_dict = {}
conflict_log = [] # 중복 데이터를 추적할 리스트

# [Step 1] 블로그 데이터를 먼저 채우기
for report in blog_data:
    sid = str(report.get('store_id'))
    merged_dict[sid] = report

# [Step 2] Kakao 리뷰 데이터로 덮어쓰며 중복 확인
duplicate_count = 0
for report in review_data:
    sid = str(report.get('store_id'))
    s_name = report.get('store_name', 'Unknown')

    if sid in merged_dict:
        # 중복 발생 시 로그 기록
        conflict_log.append({
            "ID": sid,
            "Name": s_name,
            "Action": "Overwrite (Blog -> Kakao)",
            "Blog_Review_Count": len(merged_dict[sid].get('top_keywords', [])), # 비교용 예시 데이터
            "Kakao_Review_Count": len(report.get('top_keywords', []))
        })
        duplicate_count += 1

    merged_dict[sid] = report

# 3. 최종 저장
final_integrated_list = list(merged_dict.values())
with open(final_merge_path, 'w', encoding='utf-8') as f:
    json.dump(final_integrated_list, f, ensure_ascii=False, indent=4)

# 4. 중복 리스트 상세 출력
print("="*60)
print(f"🕵️ 중복 식당 상세 확인 (총 {duplicate_count}건)")
print(f"{'ID':<15} | {'식당명':<20} | {'처리 내용'}")
print("-" * 60)

for log in conflict_log:
    print(f"{log['ID']:<15} | {log['Name']:<20} | {log['Action']}")

print("-" * 60)
print(f"✅ 최종 통합 식당 수: {len(final_integrated_list)}개")
print(f"💡 블로그 전용 데이터: {len(blog_data) - duplicate_count}개")
print("="*60)

🕵️ 중복 식당 상세 확인 (총 1건)
ID              | 식당명                  | 처리 내용
------------------------------------------------------------
1637957216      | 장터국밥                 | Overwrite (Blog -> Kakao)
------------------------------------------------------------
✅ 최종 통합 식당 수: 734개
💡 블로그 전용 데이터: 15개


In [ ]:
import json
import os

# 1. 파일 경로 설정
blog_report_path = os.path.join(base_path, 'mangwon_blog_report2.json')
review_report_path = os.path.join(base_path, 'mangwon_final_integrated_report.json')
final_merge_path = os.path.join(base_path, 'mangwon_final_final_integrated_report.json')

def load_json(path):
    if os.path.exists(path):
        with open(path, 'r', encoding='utf-8') as f:
            return json.load(f)
    return []

# 2. 데이터 로드
blog_data = load_json(blog_report_path)
review_data = load_json(review_report_path)

# 3. 병합 전략 (ID가 다르면 무조건 유지)
merged_dict = {}

# [Step 1] 블로그 데이터 먼저 등록 (93122129 포함)
for report in blog_data:
    sid = str(report.get('store_id')).strip()
    merged_dict[sid] = report

# [Step 2] 기존 통합 데이터 등록 (1637957216 포함)
collision_count = 0
collision_names = []

for report in review_data:
    sid = str(report.get('store_id')).strip()
    s_name = report.get('store_name', 'Unknown')

    # ID가 완전히 똑같은 경우만 중복으로 처리
    if sid in merged_dict:
        collision_count += 1
        collision_names.append(f"{s_name}({sid})")
        # 중복 ID일 경우 기존 데이터를 유지할지, 새 데이터를 덮어쓸지 결정
        # 여기서는 Kakao 우선 원칙에 따라 덮어씁니다.
        merged_dict[sid] = report
    else:
        # ID가 다르면 이름이 같아도 새로운 항목으로 추가됨
        merged_dict[sid] = report

# 4. 결과 저장
final_integrated_list = list(merged_dict.values())
with open(final_merge_path, 'w', encoding='utf-8') as f:
    json.dump(final_integrated_list, f, ensure_ascii=False, indent=4)

# 5. 결과 확인
print("="*60)
print(f"✅ ID 기반 독립 병합 완료")
print(f"- 최종 파일 내 식당 총 개수: {len(final_integrated_list)}개")
print(f"- 실제 ID가 충돌하여 덮어쓴 식당: {collision_count}개")
if collision_names:
    print(f"  (충돌 목록: {collision_names})")
print("-" * 60)
print("💡 확인: 이제 ID 93122129와 1637957216은 별개의 데이터로 보존됩니다.")
print("="*60)

✅ ID 기반 독립 병합 완료
- 최종 파일 내 식당 총 개수: 733개
- 실제 ID가 충돌하여 덮어쓴 식당: 1개
  (충돌 목록: ['장터국밥(1637957216)'])
------------------------------------------------------------
💡 확인: 이제 ID 93122129와 1637957216은 별개의 데이터로 보존됩니다.


In [ ]:
import json
import os
import pandas as pd

# 1. 데이터 로드
csv_path = os.path.join(base_path, 'check_list.csv')
blog_report_path = os.path.join(base_path, 'mangwon_blog_report2.json')
review_report_path = os.path.join(base_path, 'mangwon_final_integrated_report.json')
final_merge_path = os.path.join(base_path, 'mangwon_final_final_integrated_report.json')

# CSV 로드 (이름-ID 매핑용)
mapped_df = pd.read_csv(csv_path, encoding='cp949', dtype={'ID': str})
name_to_id = dict(zip(mapped_df['Name'].str.strip(), mapped_df['ID']))

def load_json(path):
    if os.path.exists(path):
        with open(path, 'r', encoding='utf-8-sig') as f:
            return json.load(f)
    return []

blog_data = load_json(blog_report_path)
review_data = load_json(review_report_path)

merged_dict = {}

# [Step 1] 블로그 데이터 처리 (ID가 없으면 CSV에서 찾아 부여)
for report in blog_data:
    raw_name = report.get('store_name', '').strip()

    # ID 찾기: 1. 기존 store_id 확인 -> 2. CSV 매핑 확인
    sid = str(report.get('store_id', ''))
    if not sid or sid == 'None' or sid == '':
        sid = name_to_id.get(raw_name, "UNKNOWN")

    if sid != "UNKNOWN":
        report['store_id'] = sid # ID 강제 주입
        merged_dict[sid] = report
        if sid == "93122129":
            print(f"✅ 망원장터국밥(93122129) 데이터 식별 및 ID 부여 성공!")

# [Step 2] 기존 Kakao 데이터 병합 (중복 시 Kakao 우선)
for report in review_data:
    sid = str(report.get('store_id')).strip()
    merged_dict[sid] = report

# [Step 3] 최종 저장
final_integrated_list = list(merged_dict.values())
with open(final_merge_path, 'w', encoding='utf-8') as f:
    json.dump(final_integrated_list, f, ensure_ascii=False, indent=4)

# [Step 4] 최종 검증
final_ids = {str(item.get('store_id')) for item in final_integrated_list}
target_id = "93122129"

print("\n" + "="*50)
print(f"📊 최종 병합 정산")
print(f"- ID {target_id} (망원장터국밥) 존재 여부: {'존재 (성공)' if target_id in final_ids else '미존재 (실패)'}")
print(f"- ID 1637957216 (장터국밥) 존재 여부: {'존재 (성공)' if '1637957216' in final_ids else '미존재 (실패)'}")
print(f"- 총 통합 식당 수: {len(final_integrated_list)}개")
print("="*50)


📊 최종 병합 정산
- ID 93122129 (망원장터국밥) 존재 여부: 미존재 (실패)
- ID 1637957216 (장터국밥) 존재 여부: 존재 (성공)
- 총 통합 식당 수: 733개


In [ ]:
import pandas as pd
import os

# 1. 파일 경로 설정
file_path = '/content/drive/MyDrive/likelion-khai/mangwon_mapped_v3.csv'

try:
    # 2. 데이터 불러오기
    df = pd.read_csv(file_path)

    # 3. 데이터 필터링 (비어있는 값 + '없음' 텍스트 제외)
    # 실제 전화번호가 있는 행만 남깁니다.
    df_filtered = df.dropna(subset=['전화번호']).copy()

    # '없음' 문자열과 공백을 모두 제외합니다.
    df_filtered = df_filtered[
        (df_filtered['전화번호'].str.strip() != '없음') &
        (df_filtered['전화번호'].str.strip() != '')
    ]

    # 4. 필터링된 데이터 중에서 전화번호 중복 추출
    duplicates = df_filtered[df_filtered.duplicated(subset=['전화번호'], keep=False)]

    # 5. 보기 좋게 정렬
    duplicates = duplicates.sort_values(by='전화번호')

    # 6. 결과 출력
    if not duplicates.empty:
        print(f"실제 전화번호가 중복된 매장 건수: {len(duplicates)}개")
        display(duplicates[['장소명', '전화번호', '주소', '카테고리']])
    else:
        print("전화번호가 기재된 매장 중 중복된 데이터가 없습니다.")

except Exception as e:
    print(f"오류가 발생했습니다: {e}")

실제 전화번호가 중복된 매장 건수: 6개


,장소명,전화번호,주소,카테고리
78,광,02-3143-6949,서울 마포구 망원동 475-51,음식점 > 일식 > 참치회
84,광참치,02-3143-6949,서울 마포구 망원동 475-51,음식점 > 일식 > 참치회
5,고향집,02-322-8762,서울 마포구 망원동 414-20,음식점 > 한식 > 국수 > 칼국수
292,고향집칼국수,02-322-8762,서울 마포구 망원동 414-67,음식점 > 한식 > 국수 > 칼국수
8,어수선,02-332-2979,서울 마포구 망원동 485-2,"음식점 > 한식 > 해물,생선 > 회"
216,어수선회&초밥,02-332-2979,서울 마포구 망원동 485-2,"음식점 > 한식 > 해물,생선 > 회"


In [ ]:
import json

# 파일 경로
json_path = '/content/drive/MyDrive/likelion-khai/mangwon_final_final_integrated_report.json'

with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 삭제할 ID 리스트 (숫자형으로 입력)
ids_to_remove = [1737054562, 415316901, 1543003311]

# 1. 데이터 필터링 (키 이름을 store_id로 지정)
original_count = len(data)

# 숫자/문자열 상관없이 store_id가 일치하면 제외
filtered_data = [
    item for item in data
    if item.get('store_id') not in ids_to_remove and str(item.get('store_id')) not in map(str, ids_to_remove)
]

new_count = len(filtered_data)

# 2. 결과 저장
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=4)

print(f"이전 매장 수: {original_count}")
print(f"삭제된 매장 수: {original_count - new_count}")
print(f"현재 매장 수: {new_count}")
print(f"삭제 완료: {ids_to_remove}")

이전 매장 수: 734
삭제된 매장 수: 3
현재 매장 수: 731
삭제 완료: [1737054562, 415316901, 1543003311]
